https://datafai.com/2018/01/19/950/
Time-series Forecasting Using Facebook Prophet Package

In [ ]:
#for colab.research.google.com
#!pip install fbprophet

In [1]:
import seaborn as sns
import pandas as pd
import warnings 
import matplotlib.pyplot as plt
#warnings.filterwarnings('ignore')

In [2]:
#from fbprophet import Prophet

In [4]:
fd=sns.load_dataset('flights')
fd.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [ ]:
fd=fd.loc[:,['passengers']]
fd.head()

# Data as a timestamp is needed by the package

In [ ]:
months=pd.date_range('19490101',periods=144,freq='M')
months
fd['ds']=months
fd.head()
fd.dtypes

In [ ]:
fd=fd.rename(columns={'passengers':'y'})
fd=fd.sort_index(axis=1)
#fd.y=np.Log(fd.y)
fd.head(5)

# Build timeseries model

In [ ]:
my_forecast=Prophet(interval_width=0.95)

In [ ]:
my_forecast.fit(fd)

In [ ]:
forecast_periods=my_forecast.make_future_dataframe(periods=36,include_history=True,freq='M')
forecast_periods

In [ ]:
forecast=my_forecast.predict(forecast_periods)
forecast1=forecast.loc[:,['ds','yhat']]
forecast1.head()

In [ ]:
my_forecast.plot(forecast,uncertainty=True)

In [ ]:
my_forecast.plot_components(forecast)

In [ ]:
final_df=pd.merge(forecast1,fd,on='ds',how='inner')
final_df.head()

# Compute Accuracy of the forecast

In [ ]:
import numpy as np
from sklearn import metrics
np.round(metrics.mean_absolute_error(final_df.yhat,final_df.y),0)

You can see from the above that on average we are off the actual numbers by around 17.
 lets see if we can improve this by taking log transformation of the 'y'

In [ ]:
fd.head()

In [ ]:
fd.y=np.log(fd.y)
fd.head()

# Build Time series model

In [ ]:
my_forecast=Prophet(interval_width=0.95)
my_forecast.fit(fd)
forecast_periods=my_forecast.make_future_dataframe(periods=36,include_history=True,freq='M')
forecast_periods
forecast=my_forecast.predict(forecast_periods)
forecast1=forecast.loc[:,['ds','yhat']]
forecast1.head()
my_forecast.plot(forecast,uncertainty=True)

In [ ]:
my_forecast.plot_components(forecast)

In [ ]:
my_forecast.plot_components(forecast)
final_df1=pd.merge(forecast1,fd,on='ds',how='inner')
final_df1.head()

#convert 'y' back to original scale

In [ ]:
final_df1.yhat=np.e**(final_df1.yhat)
final_df1.y=np.e**(final_df1.y)
final_df1.head()

# Compute Accuracy of the forecast

In [ ]:
import numpy as np
from sklearn import metrics
np.round(metrics.mean_absolute_error(final_df1.yhat,final_df1.y),0)

You can see the converting the variable to log scale to make variance more stable in the original data helped significantly in improving the forecast accuracy

In [ ]:
final_df1.head()

In [ ]:
final_df1.tail()

Generate forecast values for the next 36 months along with the 95% confidence interval

In [ ]:
forecast2=forecast.loc[144:,['ds','yhat_lower','yhat_upper']]
forecast2.yhat=np.e**(forecast2.yhat)
forecast2.yhat_lower=np.e**(forecast2.yhat_lower)
forecast2.yhat_upper=np.e**(forecast2.yhat_upper)
np.round(forecast2,0)

In [ ]:
sns.tsplot(forecast2.yhat)
plt.show()